# word2vec Model 

In [1]:
import json
import pickle
import pandas as pd
import gensim as gs
import pandas as pd
import numpy as np
import scipy as sc
import nltk
import os
from nltk.tokenize import word_tokenize as wt
from nltk.tokenize import sent_tokenize as st
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.sequence import pad_sequences
import logging
import re
import sys
import random
from collections import Counter
from tensorflow.contrib import keras
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Bidirectional, Dense,LSTM,Input,Activation,Add,TimeDistributed,\
Permute,Flatten,RepeatVector,merge,Lambda,Multiply,Reshape
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.models import Sequential,Model
from keras.optimizers import RMSprop
from keras import backend as K
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

Using TensorFlow backend.


In [3]:
#load data
histPath = ''
#load training data
with open(histPath+'train.txt', "rb") as f1, open(histPath+'val.txt', "rb") as f2, open(histPath+'test.txt', "rb") as f3: 
    trainX, trainY = pickle.load(f1)
    valX, valY = pickle.load(f2)
    testX, testY = pickle.load(f3)
#load dictionaries
with open(histPath+'word2idx_master.json', 'r') as f1, open(histPath+'idx2word_master.json', 'r') as f2:
    word2idx = json.load(f1)
    idx2word = json.load(f2)

#load embedding matrix
embeddMatrix = np.load(histPath+'embeddMatrix_word2vec_0510.npy')


In [4]:
#params for model training
seed = 209
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
LR = 1e-4
batch_size = 32

num_train_batches = len(trainX) // batch_size
num_val_samples = len(valX) + len(trainX) - batch_size*num_train_batches
num_val_batches = len(valX) // batch_size
total_entries = (num_train_batches + num_val_batches)*batch_size

#maximum length for title 
# tMaxLen = 20
tMaxLen = 250
#maximum length for abstract
aMaxLen = 250
#total maximum length
maxlen = tMaxLen + aMaxLen

batch_norm=False

embeddDim = embeddMatrix.shape[1]
nUnique = embeddMatrix.shape[0]
hidden_units= embeddDim

learning_rate = 0.002
clip_norm = 1.0
# regularizer = l2(weight_decay) if weight_decay else None

---

## I. Data Generator

In [5]:
#padding function for abstracts
def padAbstract(x, maxL = aMaxLen, dictionary = word2idx):
    n = len(x)
    if n > maxL:
        x = x[-maxL:]
        n = maxL
    return [dictionary['_']]*(maxL - n) + x + [dictionary['*']]

#build generator for model
def generator(trainX, trainY, batch_size = batch_size, 
              nb_batches = None, model = None, seed = seed):
    
    #UNDERSTAND THIS
    c = nb_batches if nb_batches else 0
    while True:
        titles = list()
        abstracts = list()
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        
        for b in range(batch_size):
            a = random.randint(0,len(trainX)-1)
            
            #random shuffling of data
            abstract = trainX[a]
            s = random.randint(min(aMaxLen,len(abstract)), max(aMaxLen,len(abstract)))
            abstracts.append(abstract[:s])
            
            title = trainY[a]
            s = random.randint(min(tMaxLen,len(title)), max(tMaxLen,len(title)))
            titles.append(title[:s])

        # undo the seeding before we yield in order not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(abstracts, titles)

#pad sequence and convert title to labels
def conv_seq_labels(abstracts, titles, nflips = None, model = None, dictionary = word2idx):
    """abstract and titles are converted to padded input vectors. Titles are one-hot encoded to labels."""
    batch_size = len(titles)
    
    
    x = [padAbstract(a)+t for a,t in zip(abstracts, titles)] 
    x = sequence.pad_sequences(x, maxlen = maxlen, value = dictionary['_'], 
                               padding = 'post', truncating = 'post')
        
    y = np.zeros((batch_size, tMaxLen, nUnique))
    for i, it in enumerate(titles):
        it = it + [dictionary['*']] + [dictionary['_']]*tMaxLen  # output does have a eos at end
        it = it[:tMaxLen]
        y[i,:,:] = np_utils.to_categorical(it, nUnique)
        
    #The 3 inputs are abstract, title starting with eos and a one-hot encoding of the title categorical variables.
    return [x[:,:aMaxLen],x[:,aMaxLen:]], y

In [6]:
#check generator
check = next(generator(trainX, trainY, batch_size = batch_size))
print(check[0][0].shape,check[0][1].shape,check[1].shape)
print("Abstract  : ", [idx2word[str(i)] for i in check[0][0][1]])
print("Title  : ", [idx2word[str(i)] for i in check[0][1][1]])

(32, 250) (32, 250) (32, 250, 32471)
Abstract  :  ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'with', 'the', 'increase', 'in', 'available', 'data', 'parallel', 'machine', 'learning', 'has', '<ign>', '<ign>', 'become', 'an', 'increasingly', 'pressing', 'problem.', 'in', 'this', 'paper', 'we', 'present', '<ign>', '<ign>', 'the', 'first', 'parallel', 'stochastic', 'gradient', 'descent', 'algorithm', 'including', 'a', '<ign>', '<ign>', 'detailed', 'anal

In [7]:
#generator for training and validation
genTrain = generator(trainX, trainY, batch_size = batch_size)
genVal =  generator(valX, valY, nb_batches = len(valX)// batch_size, batch_size = batch_size)

---

## II. Train Model

In [8]:
K.clear_session()

In [9]:
#encoder
def getModel(num_epochs, genTrain, genVal, embeddMatrix, learning_rate, clip_norm,
             encoder_shape = aMaxLen, decoder_shape = tMaxLen, 
             nUnique = nUnique, embeddDim = embeddDim, hidden_units = hidden_units):
    
    #ENCODER
    #input shape as the vector of sequence, with length padded to 250
    encoder_inputs = Input(shape = (encoder_shape, ), name = 'encoder_input')
    
    encoder_embedding = Embedding(nUnique, embeddDim, 
                                  input_length = encoder_shape, 
                                  weights = [embeddMatrix],
                                  mask_zero = True,
                                  name = 'encoder_embedd')(encoder_inputs)
    
    encoder_lstm = Bidirectional(LSTM(hidden_units, dropout_U = 0.2,
                                      dropout_W = 0.2 , return_state=True))
    
    encoder_outputs, f_h, f_c, b_h, b_c = encoder_lstm(encoder_embedding)
    
    state_hfinal=Add()([f_h, b_h])
    state_cfinal=Add()([f_c, b_c])
    
    encoder_states = [state_hfinal,state_cfinal]
        
    #DECODER
    decoder_inputs = Input(shape = (decoder_shape, ), name = 'decoder_input')
    
    decoder_embedding = Embedding(nUnique, embeddDim, 
                                  input_length = decoder_shape, 
                                  weights = [embeddMatrix],
                                  mask_zero = True,
                                  name = 'decoder_embedd')
    
    decoder_lstm = LSTM(hidden_units,return_sequences = True, return_state=True)
    
    decoder_outputs, s_h, s_c = decoder_lstm(decoder_embedding(decoder_inputs), initial_state = encoder_states)    
    decoder_dense = Dense(decoder_shape, activation='linear')
    decoder_time_distributed = TimeDistributed(Dense(nUnique,
                                                     name = 'decoder_timedistributed'))
    decoder_activation = Activation('softmax', name = 'decoder_activation')
    decoder_outputs = decoder_activation(decoder_time_distributed(decoder_outputs))
    
    #MODEL
    model = Model(inputs = [encoder_inputs,decoder_inputs], outputs = decoder_outputs) 
    rmsprop = RMSprop(lr = learning_rate, clipnorm = clip_norm)
    model.compile(loss = 'categorical_crossentropy',optimizer = rmsprop)
    return model, 0, 0
    #FIT MODEL
#     model.fit_generator(genTrain,
#                         steps_per_epoch = num_train_batches,
#                         epochs=num_epochs, 
#                         validation_data = genVal,
#                         validation_steps = num_val_batches)
    
#     #ENCODER MODEL
#     encoder_model = Model(encoder_inputs,encoder_states)
    
#     #DECODER MODEL
#     decoder_state_inputs_h = Input(shape=(hidden_units,))
#     decoder_state_inputs_c = Input(shape=(hidden_units,)) 
#     decoder_state_inputs = [decoder_state_inputs_h, decoder_state_inputs_c]
#     decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_embedding(decoder_inputs),
#                                                                      initial_state = decoder_state_inputs)
#     decoder_states = [decoder_state_h, decoder_state_c]
#     decoder_outputs = decoder_dense(decoder_outputs)
    
#     decoder_model = Model([decoder_inputs] + decoder_state_inputs,
#                           [decoder_outputs] + decoder_states)
    
    #return model, encoder_model, decoder_model

In [12]:
mod, encoder, decoder = getModel(15, genTrain, genVal, embeddMatrix, learning_rate, clip_norm)

In [11]:
# rnn.fit_generator(genTrain,
#                     steps_per_epoch = num_train_batches,
#                     epochs=15, 
#                     validation_data = genVal,
#                     validation_steps = num_val_batches)

Epoch 1/15
104/104 [==============================] - 381s 4s/step - loss: 6.7924 - val_loss: 6.4424
Epoch 2/15
104/104 [==============================] - 365s 4s/step - loss: 6.0360 - val_loss: 6.2462
Epoch 3/15
104/104 [==============================] - 367s 4s/step - loss: 5.7731 - val_loss: 6.1418
Epoch 4/15
104/104 [==============================] - 377s 4s/step - loss: 5.5584 - val_loss: 6.0337
Epoch 5/15
104/104 [==============================] - 369s 4s/step - loss: 5.3389 - val_loss: 5.9530
Epoch 6/15
104/104 [==============================] - 375s 4s/step - loss: 5.1533 - val_loss: 5.9167
Epoch 7/15
104/104 [==============================] - 370s 4s/step - loss: 5.0003 - val_loss: 5.8666
Epoch 8/15
104/104 [==============================] - 364s 4s/step - loss: 4.8257 - val_loss: 5.8629
Epoch 9/15
104/104 [==============================] - 366s 4s/step - loss: 4.6572 - val_loss: 5.8545
Epoch 10/15
104/104 [==============================] - 374s 4s/step - loss: 4.4918 - val_lo

In [12]:
#save models
mod.save_weights('rnn_weights_0508.h5')
# encoder.save('encoder.h5')
# decoder.save('decoder.h5')

In [11]:
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 250)          0                                            
__________________________________________________________________________________________________
encoder_embedd (Embedding)      (None, 250, 100)     3247100     encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, 250)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 200), (None, 160800      encoder_embedd[0][0]             
__________________________________________________________________________________________________
decoder_em

## III. Predictions

In [13]:
mod.load_weights('word2vec_weights_0510.h5')

In [23]:
def getPrediction(model, seq, idx2word, maxLen, 
                  num_iteration, greedy = True, latitude = 5):
    '''
    Prediction for a given sequence. 
    
    Parameters:
    ===========
    1)model: rnn model
    2)seq: a single abstract, should be a vector of length 250
    3)maxLen: maximum length of predicted abstract
    4)num_iteration
    5)idx2word: dictionary for index to word
    6)greedy: default to greedy search predictions, otherwise beam search
    7)latitude: for greedy search, how many top words to consider for random choice
    '''
    
    #cache list of prediction
    prediction = list()
    #initiate title to be a vector of zeros
    init = np.zeros(maxLen)
             
    #for maximum prediction length
    for i in range(num_iteration):
        #get prediction probabilities for all unique words
        predRNN = model.predict([np.reshape(seq, (1, 250)), init.reshape(1, 250)])
        
        if greedy:

            #update next title vector to be the predicted vector
            idx = np.argmax(predRNN[0, i])
            
        else:
            
            #get top number of words
            idxV = np.argsort(predRNN[0, i])[-latitude: ]
            #randomly choose from the top words
            idx = np.random.choice(idxV)
            if i == 0:
                while idx == 1:
                    idx = np.random.choice(idxV)
            else:
                while idx == prediction[i-1]:
                    idx = np.random.choice(idxV)
        
        #index to word
        word = idx2word[str(idx)]
        init[i] = idx
        #if eos tag is predicted
        #break out of loop
        if idx == 1:
            break
        prediction.append(word)
            
    return prediction

In [24]:
#check prediction 
check = testX.copy()
check = sequence.pad_sequences(check, 250, value = word2idx['_'], 
                               padding = 'pre')

In [25]:
#example of seq2seq prediction
#true title
[idx2word[str(m)] for m in testY[40]]

# true abstract
# [idx2word[str(m)] for m in check[40]]

['dual',
 'inhibitory',
 'mechanisms',
 'for',
 'definition',
 'of',
 'receptive',
 'field',
 'characteristics',
 'in',
 'a',
 'cat',
 'striate',
 'cortex']

In [31]:
check[40]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [34]:
#prediction

#     1)model: rnn model
#     2)seq: a single abstract, should be a vector of length 250
#     3)maxLen: maximum length of predicted title
#     4)idx2word: dictionary for index to word
#     5)greedy: default to greedy search predictions, otherwise beam search
#     6)latitude: for greedy search, how many top words to consider for random choice
        
check_pred = getPrediction(mod, check[40], idx2word, 250, 20)
check_pred

['neural',
 'implementation',
 'of',
 'the',
 'lateral',
 'inhibition',
 'in',
 'the',
 'vestibulo-ocular']

---

## Evaluation

In [52]:
# Relative import of rouge_evaluation
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from myeval import rouge_evaluation

In [60]:
temp_f, temp_p, temp_r = rouge_evaluation(check_pred, [idx2word[str(m)] for m in testY[40]])

In [61]:
temp_f

0.18181817719008275